In [6]:
# Library
import random
import job
import ordonnancement
import sommet
import flowshop
import time
import copy

# Greedy
- Ne prendre tout le monde : qualité médiocre
- Ne prendre que les meilleurs : bouclé sur les mêmes solutions 

In [7]:
def randomized_greedy(LIST_JOBS , NB_MACHINE ):
    """
        Return feasible solution X | X = Ordonnancement
    """
    
    X = ordonnancement.Ordonnancement(NB_MACHINE)
    random.shuffle(LIST_JOBS)
    X.ordonnancer_liste_job(LIST_JOBS)

    return (X)

# Local search : TABOU
Ici, nous utiliserons une RL : Tabu

In [18]:
Iteration=25

def meilleur_voisin(ordo, Last_visited):                
    nb_jobs=len(ordo.sequence)
    best_ordo=ordo
    for k in range(nb_jobs-1):
        for i in range(k+1,nb_jobs):
            sequence=copy.copy(ordo.sequence)
            y1=sequence[k]
            y2=sequence[i]
            sequence[k]=y2
            sequence[i]=y1
            ordo2 = ordonnancement.Ordonnancement(ordo.nombre_machines)
            ordo2.ordonnancer_liste_job(sequence)
            if best_ordo==ordo and (ordo2.sequence not in Last_visited):
                best_ordo=ordo2
            if ordo2.duree<best_ordo.duree and (ordo2.sequence not in Last_visited):
                best_ordo=ordo2
    return best_ordo


def local_search_tabu(X_greedy, LIST_JOBS, NB_MACHINE):
    #Création d'une solution de départ
    Sc= X_greedy
    
    #S_best constitue la meilleure solution trouvée
    S_best=Sc
    
    #Initialisation de la liste des dernieres solutions trouvées
    Last_visited=[Sc.sequence]
    i=0
    while i < Iteration :
        Sc=meilleur_voisin(Sc, Last_visited)
        if Sc.duree<S_best.duree:
            S_best=Sc
        
        Last_visited.append(Sc.sequence)
        if len(Last_visited)>4:
            Last_visited.pop(0)
        i+=1
    return S_best

# GRASP
- Input : 
    - alpha : [0,1] - Random seed 
    - time limit

In [9]:
# Parameters
alpha = 0.1
time_limit = 4

In [10]:
def GRASP( alpha, time_limit, LIST_JOBS, NB_MACHINE):
    
    """
        Return best solution found X* 
    """
    
    #Création d'une solution de départ
    X_best= ordonnancement.Ordonnancement(NB_MACHINE)
    X_best.ordonnancer_liste_job(LIST_JOBS)
    print('Initial solution : {} | Durée : {}'.format([x.numero for x in X_best.sequence], X_best.duree))
    
    # CPU Time
    CPU_time_start = time.time()
    CPU_time = 0
    
    while CPU_time < time_limit : 
        X_greedy = randomized_greedy(LIST_JOBS , NB_MACHINE)
        X = local_search_tabu(X_greedy,LIST_JOBS, NB_MACHINE)          # type(X) = Ordonnancement
        if X.duree < X_best.duree:
            X_best = X 
            print('New solution found : {} | Durée : {}'.format([x.numero for x in X_best.sequence], X_best.duree))
        
        # Update CPU Time
        CPU_time = time.time() - CPU_time_start
    
    return (X_best)

# Tests

In [11]:
print("JEU 1 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('data/jeu1.txt')
L = prob.liste_jobs
m = prob.nombre_machines

# Parameters
time_limit = 1

start_time = time.time()
result=GRASP(alpha, time_limit, L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])



JEU 1 :
Initial solution : [0, 1, 2, 3] | Durée : 54
----- RESULTAT-----
Temps de résolution : 1.003s
durée : 54
[0, 1, 2, 3]


In [7]:
print("JEU 2 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('data/jeu2.txt')
L = prob.liste_jobs
m = prob.nombre_machines

# Parameters
time_limit = 1

start_time = time.time()
result=GRASP(alpha, time_limit, L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

JEU 2 :
Initial solution : [0, 1, 2, 3, 4, 5, 6, 7] | Durée : 765
New solution found : [5, 2, 3, 1, 7, 4, 6, 0] | Durée : 707
New solution found : [5, 3, 1, 0, 2, 4, 6, 7] | Durée : 706
New solution found : [2, 5, 0, 3, 1, 7, 4, 6] | Durée : 704
----- RESULTAT-----
Temps de résolution : 1.004s
durée : 704
[2, 5, 0, 3, 1, 7, 4, 6]


In [30]:
print("tai01 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('/Users/corentin/Desktop/Cours/2A/programmation_dynamique/projet/data/jeu_de_données_1/tai01.txt')
L = prob.liste_jobs
m = prob.nombre_machines

# Parameters
time_limit = 300

start_time = time.time()
result=GRASP(alpha, time_limit, L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

tai01 :
Initial solution : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] | Durée : 1448
New solution found : [12, 0, 7, 10, 8, 2, 3, 5, 14, 15, 11, 18, 4, 13, 16, 1, 17, 9, 6, 19] | Durée : 1305
New solution found : [12, 16, 15, 8, 14, 5, 13, 18, 3, 4, 17, 1, 2, 6, 0, 11, 7, 10, 9, 19] | Durée : 1297
New solution found : [8, 14, 5, 16, 13, 4, 7, 3, 2, 15, 10, 12, 17, 18, 6, 0, 1, 9, 19, 11] | Durée : 1278
----- RESULTAT-----
Temps de résolution : 300.1s
durée : 1278
[8, 14, 5, 16, 13, 4, 7, 3, 2, 15, 10, 12, 17, 18, 6, 0, 1, 9, 19, 11]


In [16]:
print("tai52 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('/Users/corentin/Desktop/Cours/2A/programmation_dynamique/projet/data/jeu_de_données_2/tai52.txt')
L = prob.liste_jobs
m = prob.nombre_machines

# Parameters
time_limit = 300

start_time = time.time()
result=GRASP(alpha, time_limit, L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

tai52 :
Initial solution : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49] | Durée : 4730


KeyboardInterrupt: 

In [19]:
print("tai51 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('/Users/corentin/Desktop/Cours/2A/programmation_dynamique/projet/data/jeu_de_données_1/tai51.txt')
L = prob.liste_jobs
m = prob.nombre_machines

# Parameters
time_limit = 300

start_time = time.time()
result=GRASP(alpha, time_limit, L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

tai51 :
Initial solution : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49] | Durée : 5094
New solution found : [34, 16, 14, 26, 36, 42, 44, 5, 19, 37, 0, 30, 6, 39, 29, 45, 15, 33, 48, 7, 20, 10, 28, 41, 32, 38, 27, 46, 35, 21, 4, 13, 43, 47, 11, 9, 18, 3, 23, 17, 31, 8, 25, 1, 22, 12, 40, 24, 49, 2] | Durée : 4023
New solution found : [34, 14, 30, 37, 19, 44, 10, 11, 4, 21, 13, 41, 36, 16, 43, 38, 32, 42, 5, 31, 35, 0, 20, 46, 29, 7, 28, 1, 26, 45, 39, 3, 33, 6, 22, 47, 17, 9, 18, 23, 48, 12, 25, 15, 49, 2, 8, 40, 27, 24] | Durée : 4021
----- RESULTAT-----
Temps de résolution : 301.294s
durée : 4021
[34, 14, 30, 37, 19, 44, 10, 11, 4, 21, 13, 41, 36, 16, 43, 38, 32, 42, 5, 31, 35, 0, 20, 46, 29, 7, 28, 1, 26, 45, 39, 3, 33, 6, 22, 47, 17, 9, 18, 23, 48, 12, 25, 15, 49, 2, 8, 40, 27, 24]
